In [ ]:
import pypsa
import pandas as pd
import numpy as np
import sys
import os

PATH = "../workflows"
SCRIPTS_PATH = "pypsa-eur-sec/scripts/"

sys.path.append(os.path.join(PATH, SCRIPTS_PATH))
from helper import override_component_attrs

In [ ]:
CLUSTERS = 181
LL = "opt"
OPTS = "Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10"
RUN = "20221227-main"
SCENARIO = f"elec_s_{CLUSTERS}_lv{LL}__{OPTS}_2050"
OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

OUTPUT = "../results/graphics-20221227/"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/{SCENARIO}/"

In [ ]:
if not os.path.exists(OUTPUT_SCENARIO):
    os.makedirs(OUTPUT_SCENARIO)

In [ ]:
overrides = override_component_attrs(OVERRIDES)
fn = f"{PATH}/pypsa-eur-sec/results/{RUN}/postnetworks/{SCENARIO}.nc"
n = pypsa.Network(fn, override_component_attrs=overrides)

In [ ]:
ac = "acn" if LL == "1.0" else "acy"
hy = "hyn" if "noH2network" in OPTS else "hyy"
code = ac + hy

In [ ]:
latex = {}

In [ ]:
def correlation_electrolysis(regex):

    e = n.links.query("carrier == 'H2 Electrolysis'").set_index("bus0").p_nom_opt

    df = (
        n.generators.filter(regex=regex, axis=0)
        .groupby("bus")
        .p_nom_opt.sum()
        .reindex(n.buses.location)
    )
    s = df.groupby(df.index).sum().drop("EU")

    return np.corrcoef(e, s)[0, 1]

In [ ]:
latex[code + "correlationsolar"] = correlation_electrolysis("(solar$|solar rooftop)")

In [ ]:
latex[code + "correlationwind"] = correlation_electrolysis("(wind)")

In [ ]:
df = n.links.filter(regex="(OCGT|CHP|Fuel Cell)", axis=0)

cap = df.eval("p_nom_opt * efficiency").groupby(df.carrier).sum().div(1e3)  # GW

In [ ]:
cap

In [ ]:
latex[code + "firmcapacity"] = cap.sum()

In [ ]:
latex

In [ ]:
with open(f"../paper/{code}-stats.tex", "w") as f:
    for k, v in latex.items():
        f.write("\\newcommand{\\" + k + "}{" + str(v) + "}\n")